In [1]:
import simi

import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sentence_transformers import evaluation, losses, models, InputExample, SentenceTransformer, util as stu
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split
import torch
from tqdm.auto import tqdm, trange
tqdm.pandas()

pd.set_option('display.max_columns', 30)

RANDOM_STATE = 1

random.seed(RANDOM_STATE)

2024-01-05 11:55:51.247638: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def predict_msc(df, col, n_results, top_k, level=1):
    if top_k == 1:
        prediction = df[col].map(lambda c: Counter([m for m in c[:n_results]]).most_common(top_k)[0][0])
    else:
        prediction = df[col].map(lambda c: [t[0] for t in Counter([m for m in c[:n_results]]).most_common(top_k)])
    return prediction

def model_predict_msc(eval_models, df, n_results, top_k, level=1, msc_col="MSC"):
    res = pd.DataFrame({"actual": df[msc_col]})
    for n in eval_models.keys():
        res["prediction ({})".format(n)] = predict_msc(df, "{} ({})".format(msc_col, n), n_results, top_k, level=level)
        if top_k > 1:
            res["prediction ({})".format(n)] = pd.Series(map(lambda t: t[0] if t[0] in t[1] else t[1][0], zip(res["actual"].values, res["prediction ({})".format(n)].values)))
    res.index = df.index
    return res

def model_predict_msc_separate_params(eval_models, df, model_params, top_k, level=1, msc_col="MSC"):
    res = pd.DataFrame({"actual": df[msc_col]})
    for n in eval_models.keys():
        res["prediction ({})".format(n)] = predict_msc(df, "{} ({})".format(msc_col, n), model_params[n][0], top_k, level=level)
        if top_k > 1:
            res["prediction ({})".format(n)] = pd.Series(map(lambda t: t[0] if t[0] in t[1] else t[1][0], zip(res["actual"].values, res["prediction ({})".format(n)].values)))
    res.index = df.index
    return res

def model_f1(eval_models, df, n_results, top_k, level=1, msc_col="MSC"):
    ap = model_predict_msc(eval_models, df, n_results, top_k, level=level, msc_col=msc_col)
    res = pd.DataFrame({"f1 ({})".format(n): [f1_score(ap["actual"], ap["prediction ({})".format(n)], average="weighted")] for n in eval_models.keys()})
    return res

def model_f1_separate_params(eval_models, df, model_params, top_k, level=1, msc_col="MSC"):
    ap = model_predict_msc_separate_params(eval_models, df, model_params, top_k, level=level, msc_col=msc_col)
    res = pd.DataFrame({"f1 ({})".format(n): [f1_score(ap["actual"], ap["prediction ({})".format(n)], average="weighted")] for n in eval_models.keys()})
    return res


def model_accuracy_separate_params(eval_models, df, model_params, top_k, level=1, msc_col="MSC"):
    ap = model_predict_msc_separate_params(eval_models, df, model_params, top_k, level=level, msc_col=msc_col)
    res = pd.DataFrame({"accuracy ({})".format(n): [accuracy_score(ap["actual"], ap["prediction ({})".format(n)], normalize=True)] for n in eval_models.keys()})
    return res

def model_f1_param(eval_models, df, n_range, top_k, level=1, msc_col="MSC"):
    res = pd.concat([model_f1(eval_models, df, i, top_k, level=level, msc_col=msc_col) for i in tqdm(n_range, leave=False)], ignore_index=True)
    res.index = n_range
    return res

def best_model_f1_param(eval_models, df, n_range, top_k, level=1, msc_col="MSC"):
    params = model_f1_param(eval_models, df, n_range, top_k, level=level, msc_col=msc_col)
    best = {}
    for n in eval_models.keys():
        cn = "f1 ({})".format(n)
        print(n, list(params[cn].sort_values(ascending=False).items())[:5])
        best[n] = (params[cn].idxmax(), params[cn].max())
    return best

In [3]:
# Prerequisites for the models
bert_base = models.Transformer('bert-base-uncased')
bert_mp = SentenceTransformer(modules=[bert_base, models.Pooling(bert_base.get_word_embedding_dimension())])

mathbert_base = models.Transformer('witiko/mathberta')
mathbert_mp = SentenceTransformer(modules=[mathbert_base, models.Pooling(mathbert_base.get_word_embedding_dimension())])
mathbert_cls = SentenceTransformer(modules=[mathbert_base, models.Pooling(mathbert_base.get_word_embedding_dimension(), pooling_mode="cls")])

bert_mlm_base = models.Transformer("./bert+re-train_mlm_abstracts_arxiv")
bert_mlm_mp = SentenceTransformer(modules=[bert_mlm_base, models.Pooling(bert_mlm_base.get_word_embedding_dimension())])

sbert = SentenceTransformer('all-mpnet-base-v2')
# alternative: sbert = SentenceTransformer('all-distilroberta-v1')

specter2_base = models.Transformer('allenai/specter2_base')

# Definition of models that are evaluated
# combine models into dict
eval_models = {
    "Bert+MP": bert_mp,
    "Bert+MP+class-arx": SentenceTransformer('./bert+mean-pooling+retrain_class_arxiv'),
    "Bert+MP+class-zbm": SentenceTransformer('./bert+mean-pooling+retrain_class_zbmath'),
    
    "Mathbert+CLS": mathbert_cls,
    "Mathbert+MP+class-arx": SentenceTransformer('./mathbert+mean-pooling+retrain_class_arxiv'),
    "Mathbert+MP+class-zbm": SentenceTransformer('./mathbert+mean-pooling+retrain_class_zbmath'),
    
    "Bert+TSDAE+MP": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv'),
    "Bert+TSDAE+MP+class-arx": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv+retrain_class_arxiv'),
    "Bert+TSDAE+MP+class-zbm": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv+retrain_class_zbmath'),
    
    "Bert+MLM+MP": bert_mlm_mp,
    "Bert+MLM+MP+class-arx": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_arxiv'),
    "Bert+MLM+MP+class-zbm": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_zbmath'),
    "Bert+MLM+MP+class-zbm+anch-arx": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_zbmath_anchor_arxiv'),
    "Bert+MLM+MP+class-arx+anch-arx+class-zbm": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_arxiv_anchor_arxiv_class_zbmath'),
    
    "SBert": sbert,
    "SBert+ret_class-arx": SentenceTransformer('./sbert+retrain_class_arxiv'),
    "SBert+ret_class-zbm": SentenceTransformer('./sbert+retrain_class_zbmath'),
    
    "Specter 2+MP": SentenceTransformer(modules=[specter2_base, models.Pooling(specter2_base.get_word_embedding_dimension())]),
    "Specter 2.0+MP+class-arx": SentenceTransformer('./specter2+mp+retrain_class_arxiv'),
    "Specter 2.0+MP+class-zbm": SentenceTransformer('./specter2+mp+retrain_class_zbmath'),
    "Specter 2+MP+class-zbm+anch-arx": SentenceTransformer('./specter2+mp+retrain_class_zbmath_anchor_arxiv'),
    "Specter 2+MP+anch-arx+class-arx+class-zbm": SentenceTransformer('./specter2+mp+retrain_anchor_arxiv_class_arxiv_class_zbmath'),
    
    "SGPT": SentenceTransformer('Muennighoff/SGPT-125M-weightedmean-nli-bitfit'),
    "SGPT+class-arx": SentenceTransformer('./gpt+retrain_class_arxiv'),
    "SGPT+class-zbm": SentenceTransformer('./gpt+retrain_class_zbmath'),
}

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at witiko/mathberta were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_he

## Title--Title

In [4]:
%%time
X_test_tt = pd.read_pickle("arxiv-title-title-200-recommendations-single-class-multi-test.pkl")
X_eval_tt = pd.read_pickle("arxiv-title-title-200-recommendations-single-class-multi-eval.pkl")

print(len(X_test_tt), len(X_eval_tt))

27229 27230
CPU times: user 21.2 s, sys: 3.81 s, total: 25 s
Wall time: 25 s


In [5]:
X_test_tt.sample(5)

,title,categories,embedding (Bert+MP),embedding (Bert+MP+class-arx),embedding (Bert+MP+class-zbm),embedding (Mathbert+CLS),embedding (Mathbert+MP+class-arx),embedding (Mathbert+MP+class-zbm),embedding (Bert+TSDAE+MP),embedding (Bert+TSDAE+MP+class-arx),embedding (Bert+TSDAE+MP+class-zbm),embedding (Bert+MLM+MP),embedding (Bert+MLM+MP+class-arx),embedding (Bert+MLM+MP+class-zbm),embedding (Bert+MLM+MP+class-zbm+anch-arx),...,categories (Bert+MLM+MP+class-arx),categories (Bert+MLM+MP+class-zbm),categories (Bert+MLM+MP+class-zbm+anch-arx),categories (Bert+MLM+MP+class-arx+anch-arx+class-zbm),categories (SBert),categories (SBert+ret_class-arx),categories (SBert+ret_class-zbm),categories (Specter 2+MP),categories (Specter 2.0+MP+class-arx),categories (Specter 2.0+MP+class-zbm),categories (Specter 2+MP+class-zbm+anch-arx),categories (Specter 2+MP+anch-arx+class-arx+class-zbm),categories (SGPT),categories (SGPT+class-arx),categories (SGPT+class-zbm)
24353,Harnack Inequalities for Subordinate Brownian ...,math.PR,"[-0.38398626, -0.37396145, -0.32304454, 0.0338...","[0.1722744, 0.018208822, -1.4764434, -0.429748...","[0.052974578, 0.41982952, 0.30019072, -0.07317...","[-0.028971104, 0.021653645, -0.012819173, 0.00...","[0.14711817, 0.043924585, 0.8314516, 1.1737965...","[-0.7503378, 0.07142072, 0.3475067, 0.503294, ...","[0.042507958, 0.005969473, -0.4973062, -0.2143...","[0.24840917, -0.4662582, -0.37150866, -0.24538...","[0.7202365, 0.6132239, 0.2958774, -0.104528, 0...","[0.062142465, 0.01654627, -0.26322088, 0.09067...","[0.59962505, 0.55640644, 0.10389257, 0.3827884...","[-0.37230965, -0.04792855, -0.70057976, -0.303...","[0.2894008, -0.69465864, -0.36246306, 0.301545...",...,"[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ..."
2396,Global existence for coupled systems of nonlin...,math.AP,"[-0.37994826, -0.110305145, 0.040761195, -0.38...","[-0.36739928, -0.74158615, 0.76906145, 0.60921...","[-0.090980574, 0.48445877, 0.7358178, 0.346067...","[-0.042126313, 0.0401233, 0.0075197793, -0.000...","[0.13573292, 0.7814889, 0.4315354, 0.3727283, ...","[-0.35280666, 0.9462079, -0.48490912, -0.30962...","[-0.22328791, 0.3632737, -0.006970109, -0.0539...","[-0.84030783, -0.6146655, 0.04366744, 0.647006...","[-0.69362175, -0.03531916, 0.6799429, 0.231716...","[-0.5015558, -0.03594123, -0.062080055, 0.1179...","[0.14676462, -0.6510195, 0.029486775, 0.603843...","[-0.9622352, 0.20750482, 0.17678647, 0.7981880...","[-1.176965, -0.20007606, 0.6282327, 0.7393086,...",...,"[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, ...","[math.AP, math.AP, math.AP, math.AP, math.AP, .

In [6]:
%%time
best_tt = best_model_f1_param(eval_models, X_test_tt, list(range(5, 51)), 1, level=1, msc_col="categories")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(7, 0.5813180027142106), (8, 0.580629808323837), (6, 0.5803824958514847), (5, 0.5798086094911675), (10, 0.5785220990554026)]
Bert+MP+class-arx [(8, 0.5865911663179565), (10, 0.5861797138760078), (7, 0.585804840250271), (9, 0.5856391069999634), (13, 0.585576468811571)]
Bert+MP+class-zbm [(6, 0.6207504912750731), (5, 0.619421856601944), (8, 0.6169125413177994), (7, 0.6167212038062646), (9, 0.6156700220650662)]
Mathbert+CLS [(6, 0.5984216870284822), (7, 0.5975079155777507), (11, 0.5964746353249826), (12, 0.5962863967123351), (8, 0.596134943792338)]
Mathbert+MP+class-arx [(14, 0.5698241693025449), (16, 0.5696641711128562), (22, 0.5691779973958664), (15, 0.5691233883961767), (13, 0.5688388571780728)]
Mathbert+MP+class-zbm [(9, 0.571179654017556), (10, 0.5704851892170961), (12, 0.5701477221406828), (14, 0.5698002654485007), (8, 0.5697831293762122)]
Bert+TSDAE+MP [(6, 0.7069960059557873), (9, 0.7061701513538243), (8, 0.7061085022109598), (7, 0.705884298050544), (10, 0.705672362867127

In [7]:
model_accuracy_separate_params(eval_models, X_test_tt, best_tt, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.593301,0.597672,0.627162,0.60733,0.582651,0.581476,0.7131,0.61317,0.637409,0.726652,0.675934,0.679974,0.668515,0.694847,0.745161,0.620037,0.647912,0.722649,0.619817,0.62349,0.652576,0.675089,0.626575,0.578538,0.639575


In [8]:
model_f1_separate_params(eval_models, X_test_tt, best_tt, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.581318,0.586591,0.62075,0.598422,0.569824,0.57118,0.706996,0.5981,0.631408,0.721664,0.667692,0.674327,0.659655,0.688135,0.740433,0.611922,0.641382,0.716922,0.604479,0.614695,0.646202,0.668234,0.617863,0.567333,0.633223


In [9]:
model_accuracy_separate_params(eval_models, X_eval_tt, best_tt, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.586706,0.598274,0.627506,0.604701,0.582446,0.583878,0.708924,0.612339,0.635586,0.722696,0.672677,0.67848,0.669445,0.696217,0.74675,0.620676,0.646199,0.722145,0.619611,0.623724,0.653874,0.674844,0.62365,0.578296,0.639772


In [10]:
model_f1_separate_params(eval_models, X_eval_tt, best_tt, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.574629,0.586659,0.620627,0.594376,0.56969,0.573529,0.702405,0.597509,0.628752,0.717317,0.663597,0.672468,0.66023,0.689011,0.741567,0.612095,0.640341,0.716692,0.604473,0.61464,0.647204,0.666905,0.614117,0.566905,0.633512


## Title--Abstract

In [11]:
%%time
X_test_at = pd.read_pickle("arxiv-full-text-title-200-recommendations-single-class-multi-test.pkl")
X_eval_at = pd.read_pickle("arxiv-full-text-title-200-recommendations-single-class-multi-eval.pkl")

print(len(X_test_at), len(X_eval_at))

27229 27230
CPU times: user 18.2 s, sys: 3.86 s, total: 22.1 s
Wall time: 23.8 s


In [12]:
X_test_at.sample(5)

,text,categories,embedding (Bert+MP),embedding (Bert+MP+class-arx),embedding (Bert+MP+class-zbm),embedding (Mathbert+CLS),embedding (Mathbert+MP+class-arx),embedding (Mathbert+MP+class-zbm),embedding (Bert+TSDAE+MP),embedding (Bert+TSDAE+MP+class-arx),embedding (Bert+TSDAE+MP+class-zbm),embedding (Bert+MLM+MP),embedding (Bert+MLM+MP+class-arx),embedding (Bert+MLM+MP+class-zbm),embedding (Bert+MLM+MP+class-zbm+anch-arx),...,categories (Bert+MLM+MP+class-arx),categories (Bert+MLM+MP+class-zbm),categories (Bert+MLM+MP+class-zbm+anch-arx),categories (Bert+MLM+MP+class-arx+anch-arx+class-zbm),categories (SBert),categories (SBert+ret_class-arx),categories (SBert+ret_class-zbm),categories (Specter 2+MP),categories (Specter 2.0+MP+class-arx),categories (Specter 2.0+MP+class-zbm),categories (Specter 2+MP+class-zbm+anch-arx),categories (Specter 2+MP+anch-arx+class-arx+class-zbm),categories (SGPT),categories (SGPT+class-arx),categories (SGPT+class-zbm)
23312,Sampling methods for multistage robust convex ...,math.OC,"[-0.7331775, -0.24801154, -0.083947845, -0.170...","[-0.5865272, 0.66515946, 0.5050113, 0.21335815...","[-1.0286691, 0.8689573, -0.21069114, -0.255932...","[-0.003962959, 0.019124886, -0.016769907, -0.0...","[-0.8295391, -0.093188405, -0.29209992, -0.290...","[0.5476714, -0.8874514, 0.30851218, -0.7782917...","[0.047472883, 0.13100572, -0.25733075, -0.0063...","[-0.6472074, -0.4389135, -0.10318885, 0.810424...","[-0.95514137, -0.039616253, -0.333585, 0.19279...","[-0.4161406, -0.049836442, -0.055855077, 0.177...","[-0.79438096, 0.24737014, 0.7260278, -0.340270...","[-0.6687586, -0.73656464, 0.01400965, -0.55691...","[-0.5363895, -0.7919488, 0.005376098, -0.42696...",...,"[math.OC, math.OC, math.OC, math.OC, math.NA, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.NA, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, stat.CO, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, stat.CO, ...","[math.NA, math.NT, stat.ML, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, stat.ML, math.OC, math.NA, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ..."
23879,A Localization Algorithm for $D$-modules. We p...,math.AG,"[-0.062482703, -0.33676338, -0.05703937, -0.26...","[0.14895935, -0.25304818, -0.41364038, 0.54545...","[0.7120297, 0.3402551, -0.21290143, 0.1505873,...","[-0.022614228, 0.025036316, -0.013512723, -0.0...","[0.04508748, 0.7990777, 0.391439, 0.7352477, 0...","[0.51495916, -1.0268395, -0.25590843, 0.321771...","[0.20357072, -0.071068354, -0.3064828, 0.15304...","[-1.2828056, -0.7872235, 0.20196342, -0.223241...","[-0.19575985, -0.3267132, 0.2415732, 0.0259061...","[-0.17802233, -0.20029725, -0.048315655, 0.008...","[0.4897103, -0.82279044, -0.033569288, -0.7815...","[-0.33167458, -1.278947, 1.0546331, -1.1028546...","[-0.31605995, -0.4723723, 0.38167357, -0.19623...",...,"[math.NT, math.RT, math.RT, math.AC, math.RA, ...","[math.AG, math.AG, math.AG, math.AG, math.AG, ...","[math.AC, math.AT, math.AG, math.AG, math.RA, ...","[math.AG, math.CV, math.HO, math.AC, math.GR, ...","[math.AG, math.AC, math.RT, math.RA, math.RA, ...","[math.AG, math.NT, math.AG, math.AC, math.AG, ...","[math.AC, math.AC, math.RT, math.AG, math.RT, ...","[math.GN, math.AG, math.AC, math.AG, math.SG, ...","[math.NT, math.RT, math.RT, math.RT, math.RT, ...","[math.AC, math.RA, math.CT, math.AC, math.AT, ...","[math.RT, math.AG, math.AC, math.AG, math.NT, ...","[math.CV, math.AG, math.GR, math.OC, math.PR, ...","[math.RA, math.AG, math.DS, math.DG, math.AG, ...","[math.OC, math.GM, math.OC, math.OC, stat.ML, ...","[math.RT, math.AG, math.AT, math.LO, math.RT, .

In [13]:
%%time
best_at = best_model_f1_param(eval_models, X_test_at, list(range(5, 51)), 1, level=1, msc_col="categories")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(17, 0.3970248649665683), (16, 0.3968966404789279), (18, 0.39682145030822924), (19, 0.39622643272521774), (15, 0.39586177170833775)]
Bert+MP+class-arx [(30, 0.5667722855666577), (29, 0.5667643965256356), (31, 0.5663475673132442), (28, 0.5662090788005723), (32, 0.5659609909657377)]
Bert+MP+class-zbm [(12, 0.5814634974429564), (6, 0.5809423009611469), (8, 0.5809260735110012), (10, 0.5806143169723205), (11, 0.5805864298363629)]
Mathbert+CLS [(16, 0.20036184423522316), (10, 0.20030445647206152), (14, 0.20002056473345767), (15, 0.19994710157684809), (12, 0.199841083525137)]
Mathbert+MP+class-arx [(50, 0.5598097887380806), (49, 0.5597488869173413), (41, 0.5595580811568001), (30, 0.5594379272541185), (40, 0.5593757496906715)]
Mathbert+MP+class-zbm [(10, 0.5497952238756645), (12, 0.5490274126232468), (13, 0.5489677890368725), (11, 0.5487848948866966), (8, 0.5476826209043584)]
Bert+TSDAE+MP [(17, 0.5923896143067265), (19, 0.5914704199905338), (14, 0.5913064452162794), (18, 0.5913022956

In [14]:
model_accuracy_separate_params(eval_models, X_test_at, best_at, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.414999,0.571817,0.587535,0.247824,0.560175,0.557604,0.605384,0.592567,0.601528,0.675824,0.657718,0.661501,0.591208,0.383084,0.744684,0.589702,0.622572,0.649308,0.595799,0.593779,0.556612,0.385986,0.565573,0.522568,0.621249


In [15]:
model_f1_separate_params(eval_models, X_test_at, best_at, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.397025,0.566772,0.581463,0.200362,0.55981,0.549795,0.59239,0.584815,0.596999,0.67197,0.658697,0.657304,0.574228,0.384154,0.742109,0.58765,0.617431,0.642947,0.594826,0.586241,0.537358,0.385366,0.557015,0.521624,0.618172


In [16]:
model_accuracy_separate_params(eval_models, X_eval_at, best_at, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.406757,0.569923,0.587367,0.245979,0.562394,0.559273,0.601616,0.590856,0.600551,0.671759,0.652883,0.664451,0.593573,0.383878,0.747448,0.591664,0.62758,0.650202,0.594712,0.594124,0.556151,0.389607,0.566765,0.523944,0.620529


In [17]:
model_f1_separate_params(eval_models, X_eval_at, best_at, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.38766,0.563849,0.580685,0.198284,0.561105,0.551149,0.588071,0.583618,0.595739,0.667695,0.6534,0.659886,0.577746,0.384224,0.744684,0.58897,0.623004,0.643825,0.594198,0.586848,0.53594,0.390016,0.557873,0.523403,0.616814


## Abstract--Title

In [18]:
%%time
X_test_ta = pd.read_pickle("arxiv-abstract-200-recommendations-from-titles-single-class-multi-test.pkl")
X_eval_ta = pd.read_pickle("arxiv-abstract-200-recommendations-from-titles-single-class-multi-eval.pkl")

print(len(X_test_ta), len(X_eval_ta))

27229 27230
CPU times: user 18.1 s, sys: 3.43 s, total: 21.6 s
Wall time: 21.6 s


In [19]:
X_test_ta.sample(5)

,title,categories,embedding (Bert+MP),embedding (Bert+MP+class-arx),embedding (Bert+MP+class-zbm),embedding (Mathbert+CLS),embedding (Mathbert+MP+class-arx),embedding (Mathbert+MP+class-zbm),embedding (Bert+TSDAE+MP),embedding (Bert+TSDAE+MP+class-arx),embedding (Bert+TSDAE+MP+class-zbm),embedding (Bert+MLM+MP),embedding (Bert+MLM+MP+class-arx),embedding (Bert+MLM+MP+class-zbm),embedding (Bert+MLM+MP+class-zbm+anch-arx),...,categories (Bert+MLM+MP+class-arx),categories (Bert+MLM+MP+class-zbm),categories (Bert+MLM+MP+class-zbm+anch-arx),categories (Bert+MLM+MP+class-arx+anch-arx+class-zbm),categories (SBert),categories (SBert+ret_class-arx),categories (SBert+ret_class-zbm),categories (Specter 2+MP),categories (Specter 2.0+MP+class-arx),categories (Specter 2.0+MP+class-zbm),categories (Specter 2+MP+class-zbm+anch-arx),categories (Specter 2+MP+anch-arx+class-arx+class-zbm),categories (SGPT),categories (SGPT+class-arx),categories (SGPT+class-zbm)
16224,Orbifold construction of holomorphic vertex op...,math.QA,"[-0.21947582, 0.114362486, 0.6700599, -0.05913...","[0.041235838, -0.23890117, 0.5163955, 0.171295...","[0.1767263, 0.21465072, 1.17077, 0.43407613, -...","[-0.026293127, 0.026390405, 0.017581955, -0.04...","[0.19733988, 0.59077597, 0.51920325, 0.9954046...","[-0.29311168, -1.1987114, 0.21528101, 0.690817...","[0.063420385, 0.12067378, 0.24089961, 0.160262...","[-0.6392631, -0.30708775, 0.8012051, -0.381784...","[-0.1798792, -0.19178154, 0.6992645, 0.0407847...","[-0.19196953, 0.13888897, -0.15975392, -0.1005...","[0.71313316, -0.00244792, 0.41629016, -0.77552...","[0.1901357, -0.18725336, 0.7837503, -0.2021909...","[0.079328984, -0.12981279, 0.792899, -0.664975...",...,"[math.QA, math.QA, math.QA, math.QA, math.QA, ...","[math.RT, math.QA, math.QA, math.QA, math.QA, ...","[math.QA, math.QA, math.QA, math.RT, math.RT, ...","[math.RA, math.GR, math.RT, math.RT, math.RT, ...","[math.QA, math.QA, math.QA, math.QA, math.QA, ...","[math.QA, math.QA, math.DG, math.RA, math.DG, ...","[math.QA, math.QA, math.RT, math.QA, math.QA, ...","[math.AG, math.QA, math.QA, math.QA, math.AG, ...","[math.QA, math.QA, math.RA, math.DG, math.MG, ...","[math.QA, math.QA, math.QA, math.QA, math.QA, ...","[math.QA, math.QA, math.GR, math.QA, math.QA, ...","[math.RT, math.DS, math.AG, math.OA, math.OA, ...","[math.QA, math.QA, math.QA, math.QA, math.QA, ...","[math.RT, math.QA, math.RA, math.QA, math.QA, ...","[math.QA, math.QA, math.QA, math.QA, math.QA, ..."
7779,Cournot Maps for Intercepting Evader Evolution...,math.OC,"[-0.53559846, -0.052475106, 0.21539582, 0.0130...","[-0.5770004, 0.45219722, 0.8726508, 0.27060026...","[-0.097320475, 0.63953584, -0.9653287, -0.2993...","[-0.008554688, 0.034904394, 0.02343175, 0.0099...","[-0.7765514, -0.16235363, -0.23966801, -0.1690...","[0.7903754, -0.26123694, -0.58268726, 0.750709...","[0.04464437, -0.18080306, 0.108888626, -0.0723...","[-0.7191815, -0.65192115, 0.06915996, 0.835480...","[-0.13728684, 0.17104699, -0.23239651, 0.60364...","[-0.27632788, -0.03974913, 0.02928222, 0.09007...","[-1.1094344, 0.15287772, 0.7009958, 0.01294481...","[-0.6073379, 0.3072832, -0.9477691, 0.22806258...","[0.46634552, 0.8543999, -0.713257, 1.0882146, ...",...,"[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.AP, math.AP, math.OC, math.OC, ...","[math.GM, math.OC, math.OC, math.DS, math.LO, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.PR, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.DS, math.AP, math.OC, math.PR, math.DS, ...","[math.PR, math.PR, math.PR, stat.AP, math.PR, ...","[math.OC, math.DS, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.AP, math.OC, math.AP, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.NA, math.DS, math.AP, math.NA, math.DS, ...","[math.AP, math.OC, math.NA, math.OC, math.DS, ...","[stat.AP, math.AT, math.AP, math.AP, math.DS, .

In [20]:
%%time
best_ta = best_model_f1_param(eval_models, X_test_ta, list(range(5, 51)), 1, level=1, msc_col="categories")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(36, 0.35867320680861425), (31, 0.35861126634318125), (29, 0.3583456581363304), (37, 0.35811590523105935), (34, 0.3580760232382165)]
Bert+MP+class-arx [(23, 0.6950515646401106), (22, 0.6945443953752032), (20, 0.6943998450427916), (19, 0.6942902005965), (18, 0.694289378964318)]
Bert+MP+class-zbm [(14, 0.6280925529184807), (13, 0.6280284020082597), (19, 0.6276548244799789), (16, 0.627621535348846), (12, 0.6273328361500053)]
Mathbert+CLS [(50, 0.39526976762583554), (47, 0.3950317140284109), (49, 0.39492463389618304), (46, 0.3944683537266421), (48, 0.39418628050335636)]
Mathbert+MP+class-arx [(17, 0.7141793734471708), (19, 0.7138960170789521), (20, 0.713811845981959), (16, 0.7137639950589292), (21, 0.7136402867443238)]
Mathbert+MP+class-zbm [(17, 0.6335469686324163), (20, 0.6333530304763385), (18, 0.633067264925916), (16, 0.6330072126080901), (21, 0.6329342866883927)]
Bert+TSDAE+MP [(12, 0.7019775130606251), (11, 0.70109510550929), (16, 0.7008986670826355), (14, 0.7002460763689689

In [21]:
model_accuracy_separate_params(eval_models, X_test_ta, best_ta, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.388189,0.717838,0.639833,0.45525,0.7362,0.648279,0.711043,0.739506,0.649565,0.696463,0.788057,0.751919,0.633736,0.707958,0.812626,0.740461,0.707077,0.696316,0.747291,0.674024,0.599545,0.599324,0.619119,0.677843,0.67799


In [22]:
model_f1_separate_params(eval_models, X_test_ta, best_ta, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.358673,0.695052,0.628093,0.39527,0.714179,0.633547,0.701978,0.721441,0.636677,0.686259,0.773113,0.743717,0.627134,0.660012,0.806951,0.720635,0.698147,0.686545,0.72558,0.660639,0.594355,0.532748,0.607783,0.655167,0.668769


In [23]:
model_accuracy_separate_params(eval_models, X_eval_ta, best_ta, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.382629,0.717187,0.646713,0.452075,0.735512,0.65123,0.709769,0.743004,0.64899,0.693537,0.790121,0.752773,0.635402,0.710209,0.812413,0.739515,0.709328,0.698788,0.749871,0.672971,0.599706,0.602461,0.618032,0.67679,0.678516


In [24]:
model_f1_separate_params(eval_models, X_eval_ta, best_ta, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.352786,0.694619,0.634627,0.391346,0.71364,0.635938,0.70085,0.724245,0.635784,0.684018,0.775257,0.744827,0.627941,0.661429,0.807057,0.718711,0.699997,0.688046,0.727228,0.658674,0.594423,0.535108,0.606583,0.653764,0.669226


## Abstract--Abstract

In [25]:
%%time
X_test_aa = pd.read_pickle("arxiv-200-recommendations-single-class-multi-test.pkl")
X_eval_aa = pd.read_pickle("arxiv-200-recommendations-single-class-multi-eval.pkl")

print(len(X_test_aa), len(X_eval_aa))

27229 27230
CPU times: user 21 s, sys: 3.46 s, total: 24.5 s
Wall time: 24.5 s


In [26]:
X_test_aa.sample(5)

,text,categories,embedding (Bert+MP),embedding (Bert+MP+class-arx),embedding (Bert+MP+class-zbm),embedding (Mathbert+CLS),embedding (Mathbert+MP+class-arx),embedding (Mathbert+MP+class-zbm),embedding (Bert+TSDAE+MP),embedding (Bert+TSDAE+MP+class-arx),embedding (Bert+TSDAE+MP+class-zbm),embedding (Bert+MLM+MP),embedding (Bert+MLM+MP+class-arx),embedding (Bert+MLM+MP+class-zbm),embedding (Bert+MLM+MP+class-zbm+anch-arx),...,categories (Bert+MLM+MP+class-arx),categories (Bert+MLM+MP+class-zbm),categories (Bert+MLM+MP+class-zbm+anch-arx),categories (Bert+MLM+MP+class-arx+anch-arx+class-zbm),categories (SBert),categories (SBert+ret_class-arx),categories (SBert+ret_class-zbm),categories (Specter 2+MP),categories (Specter 2.0+MP+class-arx),categories (Specter 2.0+MP+class-zbm),categories (Specter 2+MP+class-zbm+anch-arx),categories (Specter 2+MP+anch-arx+class-arx+class-zbm),categories (SGPT),categories (SGPT+class-arx),categories (SGPT+class-zbm)
20055,Counterexamples to local Lipschitz and local H...,math.PR,"[-0.5048336, -0.042054713, 0.39014012, -0.1757...","[0.08855442, 0.018564137, -0.61789495, -0.2860...","[-0.48090857, 0.37910014, -0.5531044, -0.07997...","[-0.021207996, 0.03394219, 0.030277263, -0.010...","[0.14534126, 0.11420757, 1.067897, 1.2443584, ...","[-0.526469, 1.1543127, 0.3240982, 0.6158566, 0...","[-0.25854203, -0.08611614, 0.26286116, -0.0946...","[0.22567849, -0.48571315, -0.33120924, -0.1545...","[-0.5319887, 0.111090675, -1.2093538, 0.646758...","[-0.29447967, -0.011461275, -0.015017518, 0.11...","[0.3471799, 0.5568449, 0.19289213, 0.45840636,...","[-0.16713865, -0.28784105, -0.13297758, 0.3250...","[0.014946352, -0.30034003, -0.7701183, 1.41252...",...,"[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.DS, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ...","[math.PR, math.PR, math.PR, math.PR, math.PR, ..."
14577,On exterior moduli of quadrilaterals and speci...,math.CA,"[-0.45876256, -0.1627307, 0.0990809, -0.154051...","[-0.7930759, -0.35823193, 1.4611629, 0.3283304...","[-0.69176584, -0.06404842, 0.45934245, 0.19990...","[-0.01887618, 0.0384861, 0.03574341, 0.0076791...","[0.13285822, -0.2839066, -0.54385716, -1.05408...","[0.05868829, -0.33474264, 0.5307375, -1.224212...","[-0.24598189, 0.33493823, -0.037888218, 0.0534...","[-0.8665221, -0.23274471, 0.8817512, -0.979240...","[-0.65420645, -0.10616767, 0.8891638, 0.024346...","[-0.25749817, 0.029946065, -0.30768916, 0.1439...","[-0.3812221, 0.1375658, 0.10730625, 0.37378713...","[0.17090301, -0.17971018, -0.28514212, 0.13376...","[0.5081525, -0.5322808, 0.68533564, 0.8018771,...",...,"[math.CA, math.CA, math.CA, math.GM, math.CA, ...","[math.CA, math.CA, math.CA, math.CA, math.FA, ...","[math.MG, math.CA, math.DG, math.CV, math.GM, ...","[math.AP, math.AP, math.FA, math.CA, math.CA, ...","[math.CV, math.AP, math.CV, math.CA, math.CA, ...","[math.CA, math.CA, math.CA, math.CA, math.CA, ...","[math.CA, math.CA, math.CV, math.CA, math.CA, ...","[math.HO, math.CA, math.CA, math.CA, math.CA, ...","[math.FA, math.FA, math.CV, math.CV, math.CA, ...","[math.AP, math.CV, math.CV, math.MG, math.CA, ...","[math.CA, math.CA, math.CA, math.CA, math.CA, ...","[math.AP, math.CV, math.AP, math.FA, math.AP, ...","[math.CA, math.CA, math.CA, math.CA, math.CA, ...","[math.CV, math.CV, math.CV, math.FA, math.OC, ...","[math.CV, math.CA, math.CA, math.LO, math.NT, .

In [27]:
%%time
best_aa = best_model_f1_param(eval_models, X_test_aa, list(range(5, 51)), 1, level=1, msc_col="categories")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(8, 0.6364259123412866), (6, 0.636380527547851), (7, 0.6360602182612485), (9, 0.6357943819866209), (10, 0.635141608118012)]
Bert+MP+class-arx [(17, 0.730450928321899), (16, 0.7301470090777815), (15, 0.7301438622961111), (11, 0.729948627309962), (12, 0.7295935208142491)]
Bert+MP+class-zbm [(7, 0.6954875007564075), (9, 0.6946766280581054), (8, 0.6945143135688766), (6, 0.6944168514179363), (10, 0.6941375038126715)]
Mathbert+CLS [(9, 0.7204131988681015), (14, 0.720367981587932), (10, 0.7203597440661791), (12, 0.7200054454906983), (11, 0.719772753029133)]
Mathbert+MP+class-arx [(20, 0.746307280056441), (25, 0.7458153839745549), (14, 0.7457813083429788), (26, 0.7456507711168506), (12, 0.7455524730896108)]
Mathbert+MP+class-zbm [(10, 0.6905146727194702), (12, 0.6901029410782076), (8, 0.6900494645909079), (13, 0.6895111897332948), (14, 0.6893072483240855)]
Bert+TSDAE+MP [(8, 0.7841822079490124), (7, 0.7839902952825241), (6, 0.7838378347157882), (5, 0.7834190840556196), (9, 0.781082428

In [28]:
model_accuracy_separate_params(eval_models, X_test_aa, best_aa, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.649748,0.74226,0.702303,0.72948,0.755959,0.699512,0.790297,0.752029,0.706673,0.817033,0.803078,0.77961,0.737412,0.79026,0.846414,0.764589,0.74428,0.801866,0.764956,0.713137,0.722098,0.767711,0.714312,0.71097,0.720335


In [29]:
model_f1_separate_params(eval_models, X_test_aa, best_aa, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.636426,0.730451,0.695488,0.720413,0.746307,0.690515,0.784182,0.740759,0.698854,0.813353,0.797591,0.775191,0.731826,0.782125,0.843649,0.75694,0.738971,0.797506,0.755707,0.705652,0.715275,0.757897,0.708789,0.701976,0.714725


In [30]:
model_accuracy_separate_params(eval_models, X_eval_aa, best_aa, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.650422,0.741425,0.707932,0.726625,0.758281,0.699266,0.785714,0.755637,0.705435,0.816232,0.80404,0.781858,0.735953,0.794087,0.846934,0.764524,0.750606,0.802791,0.767976,0.717628,0.723246,0.770327,0.712229,0.710356,0.722806


In [31]:
model_f1_separate_params(eval_models, X_eval_aa, best_aa, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.636264,0.729089,0.700694,0.717817,0.747957,0.690785,0.778868,0.743818,0.696916,0.812301,0.798465,0.777773,0.730001,0.785932,0.844343,0.756113,0.744849,0.798528,0.758412,0.709977,0.715253,0.760233,0.706796,0.701947,0.717623


## Random--Abstract

In [32]:
%%time
X_test_ra = pd.read_pickle("arxiv-random-sentence-200-recommendations-single-class-multi-test.pkl")
X_eval_ra = pd.read_pickle("arxiv-random-sentence-200-recommendations-single-class-multi-eval.pkl")

print(len(X_test_ra), len(X_eval_ra))

27229 27230
CPU times: user 17.2 s, sys: 4.77 s, total: 22 s
Wall time: 25.4 s


In [33]:
X_test_ra.sample(5)

,text,categories,embedding (Bert+MP),embedding (Bert+MP+class-arx),embedding (Bert+MP+class-zbm),embedding (Mathbert+CLS),embedding (Mathbert+MP+class-arx),embedding (Mathbert+MP+class-zbm),embedding (Bert+TSDAE+MP),embedding (Bert+TSDAE+MP+class-arx),embedding (Bert+TSDAE+MP+class-zbm),embedding (Bert+MLM+MP),embedding (Bert+MLM+MP+class-arx),embedding (Bert+MLM+MP+class-zbm),embedding (Bert+MLM+MP+class-zbm+anch-arx),...,categories (Bert+MLM+MP+class-arx),categories (Bert+MLM+MP+class-zbm),categories (Bert+MLM+MP+class-zbm+anch-arx),categories (Bert+MLM+MP+class-arx+anch-arx+class-zbm),categories (SBert),categories (SBert+ret_class-arx),categories (SBert+ret_class-zbm),categories (Specter 2+MP),categories (Specter 2.0+MP+class-arx),categories (Specter 2.0+MP+class-zbm),categories (Specter 2+MP+class-zbm+anch-arx),categories (Specter 2+MP+anch-arx+class-arx+class-zbm),categories (SGPT),categories (SGPT+class-arx),categories (SGPT+class-zbm)
5850,Model-free Learning for Risk-constrained Linea...,math.OC,"[-0.5063972, -0.40300345, 0.21680136, -0.04380...","[-0.53888196, 0.43436298, 0.23010239, 0.118911...","[-0.23763295, 0.44890407, -0.12742354, 0.02545...","[-0.015269892, 0.019065548, -0.0036650377, -0....","[-0.8214102, -0.13563672, -0.21185307, -0.2106...","[-0.1570945, 0.089028515, 1.1050949, -0.402269...","[-0.25850996, -0.17301786, -0.09042377, -0.223...","[-0.7164174, -0.47092998, -0.11866773, 0.89972...","[-0.34436217, -0.013514613, 0.101466395, 0.187...","[-0.47439808, 0.07407886, 0.08182143, 0.261753...","[-0.86945075, 0.26631048, 0.721567, -0.2521457...","[0.013483878, 0.13521205, -0.43872696, -0.0430...","[-0.1855917, -0.30762398, 0.28694555, 1.010462...",...,"[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.CO, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.DS, math.OC, math.OC, ...","[math.OC, stat.AP, math.OC, math.OC, math.PR, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ...","[math.OC, math.OC, math.OC, math.OC, math.OC, ..."
20020,On the linear intersection number of graphs. T...,math.CO,"[-0.10505405, 0.02297079, 0.36353505, -0.33293...","[1.1001792, 1.4369978, -0.1385944, -0.08756361...","[0.404438, 0.37600586, 0.05516124, 0.20339254,...","[-0.0063422844, 0.013529467, -0.029373389, -0....","[-0.20109782, 0.665747, -0.68616885, -0.019664...","[-0.39727432, -0.25124136, -0.9707692, -0.7532...","[0.06882557, 0.37722784, 0.3167174, -0.0249700...","[-0.0024646947, 0.8732511, -0.062542446, -0.32...","[0.11888771, 0.2075934, -0.10236318, 0.5488428...","[-0.1603644, 0.056982096, -0.09747878, 0.00636...","[-0.14986685, 0.7714745, -0.32113504, -0.30049...","[-0.6438081, 0.33157587, -0.019488951, 0.32938...","[-0.026387399, 0.5482529, 0.92618495, 0.500391...",...,"[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.GN, math.OC, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.LO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.FA, math.NT, math.CO, math.LO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ...","[math.CO, math.CO, math.CO, math.CO, math.CO, ..

In [34]:
%%time
best_ra = best_model_f1_param(eval_models, X_test_ra, list(range(5, 51)), 1, level=1, msc_col="categories")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(15, 0.4000771573535182), (16, 0.3995896475827808), (14, 0.399373939416528), (13, 0.39926371069709393), (10, 0.3986004624877075)]
Bert+MP+class-arx [(25, 0.5343085496168343), (26, 0.5341694344737766), (27, 0.5340933103069924), (23, 0.5340403218736499), (24, 0.533995220344524)]
Bert+MP+class-zbm [(12, 0.5105504702772544), (18, 0.5098614399392182), (11, 0.5097036311080547), (17, 0.5094134145793124), (8, 0.509390881955984)]
Mathbert+CLS [(44, 0.35509455654072153), (48, 0.35470040866054836), (45, 0.3546397663218487), (46, 0.35463654265566247), (43, 0.3546294223498131)]
Mathbert+MP+class-arx [(25, 0.548637967400663), (23, 0.5481216816599357), (22, 0.5479683796138282), (26, 0.5479289628136886), (20, 0.5478296534836049)]
Mathbert+MP+class-zbm [(14, 0.504215899885688), (15, 0.5037959065100236), (18, 0.5035592359706518), (16, 0.5035477772178047), (22, 0.5035015260007696)]
Bert+TSDAE+MP [(14, 0.5726963735632634), (13, 0.5725631740567784), (15, 0.5710915786394444), (12, 0.571003365561713

In [35]:
model_accuracy_separate_params(eval_models, X_test_ra, best_ra, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.414778,0.531492,0.514084,0.370891,0.540453,0.507474,0.57604,0.549928,0.529436,0.618311,0.606008,0.604833,0.577987,0.393735,0.668405,0.542326,0.55698,0.603107,0.55507,0.527232,0.56326,0.4013,0.498696,0.507217,0.54071


In [36]:
model_f1_separate_params(eval_models, X_test_ra, best_ra, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.400077,0.534309,0.51055,0.355095,0.548638,0.504216,0.572696,0.551693,0.529031,0.62217,0.618927,0.605402,0.57139,0.393432,0.672101,0.548414,0.556468,0.603111,0.563804,0.526651,0.554173,0.39636,0.492331,0.514022,0.539687


In [37]:
model_accuracy_separate_params(eval_models, X_eval_ra, best_ra, 1, level=1, msc_col="categories")

,accuracy (Bert+MP),accuracy (Bert+MP+class-arx),accuracy (Bert+MP+class-zbm),accuracy (Mathbert+CLS),accuracy (Mathbert+MP+class-arx),accuracy (Mathbert+MP+class-zbm),accuracy (Bert+TSDAE+MP),accuracy (Bert+TSDAE+MP+class-arx),accuracy (Bert+TSDAE+MP+class-zbm),accuracy (Bert+MLM+MP),accuracy (Bert+MLM+MP+class-arx),accuracy (Bert+MLM+MP+class-zbm),accuracy (Bert+MLM+MP+class-zbm+anch-arx),accuracy (Bert+MLM+MP+class-arx+anch-arx+class-zbm),accuracy (SBert),accuracy (SBert+ret_class-arx),accuracy (SBert+ret_class-zbm),accuracy (Specter 2+MP),accuracy (Specter 2.0+MP+class-arx),accuracy (Specter 2.0+MP+class-zbm),accuracy (Specter 2+MP+class-zbm+anch-arx),accuracy (Specter 2+MP+anch-arx+class-arx+class-zbm),accuracy (SGPT),accuracy (SGPT+class-arx),accuracy (SGPT+class-zbm)
0,0.416159,0.529747,0.516967,0.370547,0.538707,0.507271,0.575542,0.550826,0.527176,0.617444,0.600551,0.60022,0.576937,0.396511,0.67242,0.544179,0.554242,0.601763,0.555013,0.525119,0.563974,0.399119,0.499376,0.503783,0.544216


In [38]:
model_f1_separate_params(eval_models, X_eval_ra, best_ra, 1, level=1, msc_col="categories")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.401749,0.532329,0.513086,0.352821,0.547439,0.503021,0.57098,0.552293,0.526264,0.620559,0.613988,0.600769,0.570944,0.395087,0.675564,0.549085,0.552877,0.600555,0.5636,0.524115,0.554265,0.394308,0.49283,0.509208,0.542574
